# Object Avoidance and Human Following

This program will detect and follow a human whilst maintaing a safe distance. The ssd mobilenet v2 nerual network structure is used for human detection and object avoidance.



### Step 1 load the pre-trained object detection model

**Before running the program, we need to download the pre-trained detection model and put it in the current folder.** The model can be found in the following link:https://drive.google.com/file/d/1KjlDMRD8uhgQmQK-nC2CZGHFTbq4qQQH/view

In [1]:
## Import Libraries
# Source [1]
import tensorrt as trt
from tensorrt_model import TRTModel
from ssd_tensorrt import load_plugins, parse_boxes,TRT_INPUT_NAME, TRT_OUTPUT_NAME
import ctypes
import numpy as np
import cv2
import os
import ctypes
    
mean = 255.0 * np.array([0.5, 0.5, 0.5])
stdev = 255.0 * np.array([0.5, 0.5, 0.5])

## Define all the essential functions 
def bgr8_to_ssd_input(camera_value):
    ''''''
    x = camera_value
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1)).astype(np.float32)
    x -= mean[:, None, None]
    x /= stdev[:, None, None]
    return x[None, ...]

class ObjectDetector(object):
    
    def __init__(self, engine_path, preprocess_fn=bgr8_to_ssd_input):
        logger = trt.Logger()
        trt.init_libnvinfer_plugins(logger, '')
        load_plugins()
        self.trt_model = TRTModel(engine_path, input_names=[TRT_INPUT_NAME],output_names=[TRT_OUTPUT_NAME, TRT_OUTPUT_NAME + '_1'])
        self.preprocess_fn = preprocess_fn
        
    def execute(self, *inputs):
        trt_outputs = self.trt_model(self.preprocess_fn(*inputs))
        return parse_boxes(trt_outputs)
    
    def __call__(self, *inputs):
        return self.execute(*inputs)

model = ObjectDetector('ssd_mobilenet_v2_coco.engine')


### Step 2 Initialize the camera instance for the Intel realsense sensor D435i

In [2]:
#use traitlets and widgets to display the image in Jupyter Notebook
import traitlets
from traitlets.config.configurable import SingletonConfigurable

#use opencv to covert the depth image to RGB image for displaying purpose
import cv2
import numpy as np

#using realsense to capture the color and depth image
import pyrealsense2 as rs

#multi-threading is used to capture the image in real time performance
import threading

class Camera(SingletonConfigurable):
    
    # This changing of this value will be captured by traitlets
    color_value = traitlets.Any()
    
    def __init__(self):
        '''
        Configure the color and depth sensor
        Set resolution for the color camera
        Set resolution for the depth camera
        Flag to control the thread
        Start the RGBD sensor
        Start capture the first color image
        Start capture the first depth image
        '''
        super(Camera, self).__init__()
        
        #configure the color and depth sensor
        self.pipeline = rs.pipeline()
        self.configuration = rs.config()  
        
        #set resolution for the color camera
        self.color_width = 640
        self.color_height = 480
        self.color_fps = 30
        self.configuration.enable_stream(rs.stream.color, self.color_width, self.color_height, rs.format.bgr8, self.color_fps)

        #set resolution for the depth camera
        self.depth_width = 640
        self.depth_height = 480
        self.depth_fps = 30
        self.configuration.enable_stream(rs.stream.depth, self.depth_width, self.depth_height, rs.format.z16, self.depth_fps)

        #flag to control the thread
        self.thread_runnning_flag = False
        
        #start the RGBD sensor
        self.pipeline.start(self.configuration)
        self.pipeline_started = True
        frames = self.pipeline.wait_for_frames()

        #start capture the first color image
        color_frame = frames.get_color_frame()   
        image = np.asanyarray(color_frame.get_data())
        self.color_value = image

        #start capture the first depth image
        depth_frame = frames.get_depth_frame()           
        depth_image = np.asanyarray(depth_frame.get_data())
        depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.03), cv2.COLORMAP_JET)
        self.depth_value = depth_colormap   

    def _capture_frames(self):
        '''This function is used for capturing the frames and processing them'''
        while(self.thread_runnning_flag==True): #continue until the thread_runnning_flag is set to be False
            frames = self.pipeline.wait_for_frames() #receive data from RGBD sensor
            
            color_frame = frames.get_color_frame() #get the color image
            image = np.asanyarray(color_frame.get_data()) #convert color image to numpy array
            self.color_value = image #assign the numpy array image to the color_value variable 

            depth_frame = frames.get_depth_frame() #get the depth image           
            depth_image = np.asanyarray(depth_frame.get_data()) #convert depth data to numpy array
            
            
            depth_image[:190,:]=0
            depth_image[290:,:]=0
            depth_image[:,:160]=0
            depth_image[:,480:]=0
            
            #For object avoidance, we don't consider the distance that are lower than 100mm or bigger than 1000mm
            depth_image[depth_image<100]=0
            depth_image[depth_image>1000]=0
            
            #If all of the values in the depth image is 0, the depth[depth!=0] command will fail
            #we set a specific value here to prevent this failure
            depth_image[0,0]=2000
            self.depth = depth_image
            #conver depth data to BGR image for displaying purpose
            depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.03), cv2.COLORMAP_JET)
            self.depth_value = depth_colormap #assign the color BGR image to the depth value
    
    def start(self):
        ''' Start the data capture thread'''
        if self.thread_runnning_flag == False: #only process if no thread is running yet
            self.thread_runnning_flag=True #flag to control the operation of the _capture_frames function
            self.thread = threading.Thread(target=self._capture_frames) #link thread with the function
            self.thread.start() #start the thread

    def stop(self): 
        '''Stop the data capture thread'''
        if self.thread_runnning_flag == True:
            self.thread_runnning_flag = False #exit the while loop in the _capture_frames
            self.thread.join() #wait the exiting of the thread       

def bgr8_to_jpeg(value):
    '''Convert numpy array to jpeg coded data for displaying '''
    return bytes(cv2.imencode('.jpg',value)[1])

#create a camera object
camera = Camera.instance()
camera.start() # start capturing the data


### Step 3 Perform object detection and display the results on widgets
Human is labeled is 1 in the pretrained model. A full list of the detection class indices can be found in the following link https://github.com/tensorflow/models/blob/master/research/object_detection/data/mscoco_complete_label_map.pbtxt

The following program will ask the robot to follow if a human is detected in the captured color image. It will follow this human whilst maintaining a safe distance and avoiding collision with humans and objects.

In [3]:
## Took Massive help from https://github.com/NVIDIA-AI-IOT/jetbot/blob/master/notebooks/object_following/live_demo.ipynb
## Import Libraries 
import ipywidgets.widgets as widgets
from IPython.display import display, HTML
import time
from RobotClass import Robot
# Initialize the Robot class
robot = Robot()
robot_speed = 0.4
turning_speed = 2
## set frame size and wiget size
width = 640
height = 480
image_widget = widgets.Image(format='jpeg', width=300, height=300)
label_widget = widgets.IntText(value=1, description='tracked label')
stop_button = widgets.Button(description= 'Stop robot')
   
speed_widget = widgets.FloatSlider(value=0.4, min=0.0, max=1.0, description='speed')
turn_gain_widget = widgets.FloatSlider(value=0.8, min=0.0, max=2.0, description='turn gain')

def on_button_clicked():
    '''Stop robot when user presses the button'''
    robot.stop()

stop_button.on_click(on_button_clicked)

Now we define all the functions responsible for the object detection. We are choosing the detected human with the largest bounding box. 

In [4]:
def detection_center(detection):
        """
        Computes the center x, y coordinates of the object
        Parameters: matching detection
        Return: Center coordinates of the detection
        """
        bbox = detection['bbox']
        center_x = (bbox[0] + bbox[2]) / 2.0 - 0.5
        center_y = (bbox[1] + bbox[3]) / 2.0 - 0.5
        return (center_x, center_y)

def detection_area(detection):
        """
        Computes the area of the bbox of the object"
        Parameters: matching detection
        Return: Area of the bounding box of the detection
        """
        bbox = detection['bbox']
        center_x = (bbox[2] - bbox[0]) 
        center_y = (bbox[3] - bbox[1])
        return (center_x * center_y)
    
def closest_detection(detections):
        """
        Finds the detection with the largest bbox area""
        Parameters: matching detection
        Return: Detection with the largest bounding 
        """    
        closest_detection = None
        for det in detections:
                center = detection_center(det)
                area= detection_area(det)
                if closest_detection is None:
                        closest_detection = det
                elif (detection_area(det)) > (detection_area(closest_detection)):
                        closest_detection = det
        return closest_detection

Now we do the main processing. Here the robot follows the detected human with the largest bounding box. We have also implemented two speeds such that the robot increases its speed if the human is at a further distance. For collision avoidance the robot moves backwards if it detects an object with 200mm.

In [5]:
# Displays the camera stream
display(widgets.VBox([
        widgets.HBox([image_widget]),
        label_widget,
        speed_widget,
        turn_gain_widget
    ]))
def processing(change):
    '''This part is taking the data from all the fucntions above and making the decisions to move the robot in a particular direction'''
    image = change['new']
     
    imgsized= cv2.resize(image,(300,300))
    
    # compute all detected objects
    detections = model(imgsized)
    
    matching_detections = [d for d in detections[0] if d['label'] == int(label_widget.value)]
    
   # Create a blue bbox on the camera widget for each matching detection
    for det in matching_detections:
            bbox = det['bbox']
            cv2.rectangle(image, (int(width * bbox[0]), int(height * bbox[1])), (int(width * bbox[2]), int(height * bbox[3])), (255, 0, 0), 2)

    # Create a green bbox for the matching detection with the largest detection
    det = closest_detection(matching_detections)
    if det is not None:
        bbox = det['bbox']
        cv2.rectangle(image, (int(width * bbox[0]), int(height * bbox[1])), (int(width * bbox[2]), int(height * bbox[3])), (0, 255, 0), 5)
    
    # If no human is detected the robot will turn right until it finds one
    if det is None:
            robot.stop()
            robot.right(0.5)
            time.sleep(0.3)

    # Otherwise move towards the detected human
    else:
        # Robot will move backwards if the human is too close to maintain a safe distance     
        if (camera.depth[camera.depth!=0].min()<200):
            robot.stop()
            robot.backward(0.3)
            time.sleep(0.3)
        
        # The robot stops at 700mm to maintain a safe distance and tracks human position.
        elif (camera.depth[camera.depth!=0].min()<700):
            center = detection_center(det)
            robot.stop()
            # If the human is on thr right side of the frame, the robot will turn right.            
            if center[0]> 0.2:    
                robot.right(0.4)
                time.sleep(0.3)
                robot.stop()
            elif center[0]<-0.2: # If the human is on the left side of the frame, the robot will turn left.            
                robot.left(0.4)
                time.sleep(0.3)
                robot.stop()        
        
        # Increase the speed if the distance from object is too far
        elif (camera.depth[camera.depth!=0].min()>1200):
            center = detection_center(det)
            # Adjusts the motor values to move towards the center of the detected human
            # In order to achivce maximum speed 0.6 has be added as 0.4 is the default value.
            robot.set_motors(
            float(speed_widget.value + 0.6 + turn_gain_widget.value * center[0]),
            float(speed_widget.value + 0.6 - turn_gain_widget.value * center[0]),
            float(speed_widget.value + 0.6 + turn_gain_widget.value * center[0]),
            float(speed_widget.value + 0.6 - turn_gain_widget.value * center[0])
            )

        else:
                # Go forward with normal speed.
            center = detection_center(det)
                
                # Adjusts the motor values to move towards the center of the detected human
            robot.set_motors(
            float(speed_widget.value + turn_gain_widget.value * center[0]),
            float(speed_widget.value - turn_gain_widget.value * center[0]),
            float(speed_widget.value + turn_gain_widget.value * center[0]),
            float(speed_widget.value - turn_gain_widget.value * center[0])
            )


    image_widget.value = bgr8_to_jpeg(image)
    
#the camera.observe function will monitor the color_value variable. If this value changes, the excecute function will be excuted.
camera.observe(processing, names='color_value')

Now just Stop the camera and robot.

In [ ]:
time.sleep(500)
camera.unobserve_all()
camera.stop()
robot.stop()

References:

source[1]:https://learn.lboro.ac.uk/mod/resource/view.php?id=983989

source[2]:https://github.com/NVIDIA-AI-IOT/jetbot/blob/master/notebooks/object_following/live_demo.ipynb